# Physics-Informed Neural Network

Nome:

Neste notebook resolveremos a equação de Poisson em duas dimensões usando uma rede neural informada por física

A implementação será feita da mesma forma que para a equação de Burgers, resolvida em sala.

Consulte os slides da aula 7 para a teoria e a definição do problema em mais detalhes.

#Importar bibliotecas

Nesse caso, usaremos o PyTorch para poder utilizar a funcionalidade de diferenciação automática

In [ ]:
!pip install plotly --quiet

import torch
import torch.nn as nn
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

np.random.seed(1)

# Geração do domínio

Nesta seção, geraremos os pontos que a rede usará como referência para encontrar a solução

## Exercício 1: Criar função que gera pontos no contorno do domínio

Primeiro, vamos gerar pontos ao longo da condição de contorno. Nestes pontos, sabemos qual deve ser o valor de z. Eles servem para "ancorar" o resto da superfície da solução.

O domínio tem 4 lados, cada um deles terá 1/4 do número total de pontos.

As condições de contorno são as seguintes:

Lado 1: $z(x=0,y) = 0$

Lado 2: $z(x,y=l_y) = -\sin(\pi x/l_x)$

Lado 3: $z(x=l_x,y) = -sin(2 \pi y/l_y)$

Lado 4: $z(x,y=0) = \sin(\pi x/l_x)$

Note que: $l_x$ e $l_y$ são comprimento_x e comprimento_y, respectivamente.

In [ ]:
def gerar_pontos_contorno(pontos_no_contorno,comprimento_x,comprimento_y):
  pontos_por_lado = pontos_no_contorno//4

  # Lado 1 (x = 0, qualquer y)
  # PARA VOCÊ FAZER (3 LINHAS)
  # -------------------------------
  x_lado1 =
  y_lado1 =
  z_lado1 =
  # -------------------------------
  # ATÉ AQUI

  # Lado 2 (qualquer x, y = comprimento_y)
  # PARA VOCÊ FAZER (3 LINHAS)
  # -------------------------------

  # -------------------------------
  # ATÉ AQUI

  # Lado 3 (x = comprimento_x, qualquer y)
  # PARA VOCÊ FAZER (3 LINHAS)
  # -------------------------------

  # -------------------------------
  # ATÉ AQUI

  # Lado 4 (qualquer x, y = 0)
  # PARA VOCÊ FAZER (3 LINHAS)
  # -------------------------------

  # -------------------------------
  # ATÉ AQUI

  # Juntar todos os lados
  x_todos = np.vstack((x_lado1,x_lado2,x_lado3,x_lado4))
  y_todos = np.vstack((y_lado1,y_lado2,y_lado3,y_lado4))
  z_todos = np.vstack((z_lado1,z_lado2,z_lado3,z_lado4))

  # Criar arrays X e Y
  X_contorno = np.hstack((x_todos,y_todos))
  Y_contorno = z_todos

  return X_contorno, Y_contorno

In [ ]:
# Testar a função acima com 40 pontos
X_contorno, Y_contorno = gerar_pontos_contorno(40,1,1)

print(np.hstack((X_contorno, Y_contorno)))

Saída esperada:
Cada linha aqui representa um ponto a ser usado na avaliação da condição de contorno. A primeira coluna é a posição no eixo x, a segunda, no eixo y, e a terceira, no z.
```
[[ 0.00000000e+00  4.17022005e-01  0.00000000e+00]
 [ 0.00000000e+00  7.20324493e-01  0.00000000e+00]
 [ 0.00000000e+00  1.14374817e-04  0.00000000e+00]
 [ 0.00000000e+00  3.02332573e-01  0.00000000e+00]
 [ 0.00000000e+00  1.46755891e-01  0.00000000e+00]
 [ 0.00000000e+00  9.23385948e-02  0.00000000e+00]
 [ 0.00000000e+00  1.86260211e-01  0.00000000e+00]
 [ 0.00000000e+00  3.45560727e-01  0.00000000e+00]
 [ 0.00000000e+00  3.96767474e-01  0.00000000e+00]
 [ 0.00000000e+00  5.38816734e-01  0.00000000e+00]
 [ 4.19194514e-01  1.00000000e+00 -9.67950749e-01]
 [ 6.85219500e-01  1.00000000e+00 -8.35428567e-01]
 [ 2.04452250e-01  1.00000000e+00 -5.99043233e-01]
 [ 8.78117436e-01  1.00000000e+00 -3.73617011e-01]
 [ 2.73875932e-02  1.00000000e+00 -8.59345411e-02]
 [ 6.70467510e-01  1.00000000e+00 -8.59993456e-01]
 [ 4.17304802e-01  1.00000000e+00 -9.66442756e-01]
 [ 5.58689828e-01  1.00000000e+00 -9.83050194e-01]
 [ 1.40386939e-01  1.00000000e+00 -4.26878888e-01]
 [ 1.98101489e-01  1.00000000e+00 -5.82949567e-01]
 [ 1.00000000e+00  8.00744569e-01  9.49600451e-01]
 [ 1.00000000e+00  9.68261576e-01  1.98099292e-01]
 [ 1.00000000e+00  3.13424178e-01 -9.21641810e-01]
 [ 1.00000000e+00  6.92322616e-01  9.35049489e-01]
 [ 1.00000000e+00  8.76389152e-01  7.00908084e-01]
 [ 1.00000000e+00  8.94606664e-01  6.14857974e-01]
 [ 1.00000000e+00  8.50442114e-02 -5.09280500e-01]
 [ 1.00000000e+00  3.90547832e-02 -2.42933147e-01]
 [ 1.00000000e+00  1.69830420e-01 -8.75792872e-01]
 [ 1.00000000e+00  8.78142503e-01  6.93008081e-01]
 [ 9.83468338e-02  0.00000000e+00  3.04073466e-01]
 [ 4.21107625e-01  0.00000000e+00  9.69442663e-01]
 [ 9.57889530e-01  0.00000000e+00  1.31908386e-01]
 [ 5.33165285e-01  0.00000000e+00  9.94576941e-01]
 [ 6.91877114e-01  0.00000000e+00  8.23751535e-01]
 [ 3.15515631e-01  0.00000000e+00  8.36695628e-01]
 [ 6.86500928e-01  0.00000000e+00  8.33209270e-01]
 [ 8.34625672e-01  0.00000000e+00  4.96479824e-01]
 [ 1.82882773e-02  0.00000000e+00  5.74227135e-02]
 [ 7.50144315e-01  0.00000000e+00  7.06786121e-01]]
```



## Exercício 2 - Criar função que gera pontos de avaliação da equação

Esta é a segunda classe de pontos que usaremos. Neles, não sabemos a solução. Mas sabemos qual equação eles devem obedecer.

Estes pontos estão uniformemente distribuídos ao longo do domínio.

In [ ]:
def gerar_pontos_equacao(pontos_no_dominio,comprimento_x,comprimento_y):

  # Gerar pontos aleatórios em x e y entre 0 e comprimento_x e 0 e comprimento_y, respectivamente
  # PARA VOCÊ FAZER (2 LINHAS)
  # -------------------------------
  x_dominio = 
  y_dominio = 
  # -------------------------------
  # ATÉ AQUI

  # Juntar as coordenadas x e y em um único array X
  # PARA VOCÊ FAZER (3 LINHAS)
  # -------------------------------
  X_equacao = 
  # -------------------------------
  # ATÉ AQUI

  return X_equacao

In [ ]:
# Testar a função acima com 20 pontos
X_equacao= gerar_pontos_equacao(20,1,1)

print(X_equacao)

Saída esperada:
Cada linha aqui representa um ponto a ser usado na avaliação da equação. A primeira coluna é a posição no eixo x e a segunda, no eixo y.

## Gerar pontos e plotar

Agora, vamos usar as funções que montamos acima para gerar os pontos.

A variável ressorteio controla se os pontos de avaliação da euqação serão ou não ressorteados após cada época.

Como forma de mostrar outras das ferramentas disponíveis no Python, vou usar a biblioteca Plotly em vez da Matplotlib. Minha opinião é de que é uma biblioteca mais complicada de se utilizar, mas traz o ponto positivo de facilitar a interatividade com as imagens.

In [ ]:
comprimento_x = 1
comprimento_y = 1

pontos_no_contorno = 400
pontos_no_dominio = 1000
ressorteio = True

X_contorno, Y_contorno = gerar_pontos_contorno(pontos_no_contorno,comprimento_x,comprimento_y)
X_equacao = gerar_pontos_equacao(pontos_no_dominio,comprimento_x,comprimento_y)


Plotamos uma vista superior do domínio para ver se os pontos estão nos lugares corretos

In [ ]:
# Vista superior
scatter_contorno = px.scatter(x=X_contorno[:,0],y=X_contorno[:,1])
scatter_equacao = px.scatter(x=X_equacao[:,0],y=X_equacao[:,1], color_discrete_sequence=['red'])
fig = go.Figure(data=scatter_contorno.data+scatter_equacao.data)
fig.update_layout(xaxis_title='x',yaxis_title='y')
fig.show()

E uma vista em perspectiva apenas dos pontos do contorno, para ver se parece correto

In [ ]:
# Vista em perspectiva
scatter_3d = px.scatter_3d(x=X_contorno[:,0].flatten(),y=X_contorno[:,1].flatten(),z=Y_contorno.flatten())
fig = go.Figure(scatter_3d)
fig.update_layout(scene=dict(aspectratio=dict(x=1.5, y=1.5, z=0.5)))
fig.show()

# Definição da Rede Neural

Neste seção, vamos definir a rede neural. Primeiro, montamos sua estrutura, depois definimos a função de perda e o otimizador.

## Exercício 3 - Estrutura da rede neural

Vamos criar uma função que monta uma rede neural totalmente conectada baseada em uma lista com o número de neurônios em cada camada.

Note que removemos a função de ativação da última camada pois estamos fazendo um ajuste de função

In [ ]:
def criar_rede_neural(numero_de_neuronios):

  # Criar uma lista de todas as camadas
  camadas = []

  # Para cada camada, adicionar as conexões e a função de ativação
  for i in range(len(numero_de_neuronios)-1):
    # PARA VOCÊ FAZER (2 LINHAS)
    # Adicionar uma camada de conexões lineares e, depois, uma de funções de ativação de tangente hiperbólica
    # -------------------------------

    # -------------------------------
    # ATÉ AQUI

  # Remover a última camada, pois é a função de ativação
  camadas.pop()

  # Criar rede
  return nn.Sequential(*camadas)

Agora, definimos o número de neurônios por camada e chamamos a função para inicializar a rede.

Note que a primeira camada deve ter dois neurônios, pois a função tem duas entradas.

De forma similar, a última camada deve ter apenas um neurônio, pois a função tem apenas uma saída.

In [ ]:
# PARA VOCÊ FAZER (1 LINHA)
# Definir que a RNA terá 3 camadas intermediárias, de 20 neurônios cada
# -------------------------------
numero_de_neuronios = 
# -------------------------------
# ATÉ AQUI

rna = criar_rede_neural(numero_de_neuronios)

print(rna)

Saída esperada:

```
Sequential(
  (0): Linear(in_features=2, out_features=20, bias=True)
  (1): Tanh()
  (2): Linear(in_features=20, out_features=20, bias=True)
  (3): Tanh()
  (4): Linear(in_features=20, out_features=20, bias=True)
  (5): Tanh()
  (6): Linear(in_features=20, out_features=1, bias=True)
)
```



## Definição das funções de perda

Nesta forma de implementar um PINN, precisamos de duas funções de perda diferentes. Uma será responsável pelas condições de contorno. A outra, pela observância das equações.

A perda responsável pelas condições de contorno funciona de maneira bem similar a uma rede neural convencional:
Comparamos a previsão da rede em cada ponto com o valor que temos de referência para estes pontos.

In [ ]:
def calc_perda_contorno(rna,X_contorno,Y_contorno):
  Y_predito = rna(X_contorno)
  return nn.functional.mse_loss(Y_predito, Y_contorno)

Já a perda responsável pela observância das equações funciona de forma bem diferente:

Usaremos a funcionalidade de derivadas automáticas (autograd) para obter as derivadas parciais das saídas da rede em função das entradas. Essas derivadas parciais serão usadas para se calcular o resíduo da equação em cada um dos pontos.

## Exercício 5 - Implementar equação governante

A equação governate é devinida por

$\frac{\partial^2 z}{\partial x^2} + \frac{\partial^2 z}{\partial y^2} = 0$

Dica:
Para o cálculo de derivadas de segunda ordem, basta calcular a derivada da derivada, da seguinte forma:

$z_{x} = \frac{\partial z}{\partial x}$

$z_{xx} = \frac{\partial^2 z}{\partial x^2} = \frac{\partial z_x}{\partial x}$

In [ ]:
def calc_residuo(rna,X_equacao):
  x = X_equacao[:,0].reshape(-1, 1)
  y = X_equacao[:,1].reshape(-1, 1)

  z = rna(torch.hstack((x,y)))

  # PARA VOCÊ FAZER (4 LINHAS)
  # Cálculo das derivadas necessárias para a equação governante
  # -------------------------------

  # -------------------------------
  # ATÉ AQUI

  # PARA VOCÊ FAZER (1 LINHA)
  # Cálculo do resíduo da equação governante
  # -------------------------------
  residuo = 
  # -------------------------------
  # ATÉ AQUI

  return residuo

In [ ]:
def calc_perda_equacao(rna,X_equacao):

  residuo = calc_residuo(rna,X_equacao)

  return torch.mean(torch.square(residuo))

In [ ]:
def calc_perda(rna,X_contorno,Y_contorno,X_equacao,alpha=0.1):

  perda_contorno = calc_perda_contorno(rna,X_contorno,Y_contorno)
  perda_equacao = calc_perda_equacao(rna,X_equacao)

  perda = (1-alpha)*perda_contorno + alpha*perda_equacao

  return perda, perda_contorno, perda_equacao

## Exercício 6 - Definir otimizador

No PyTorch, uma das entradas para o otimizador são os parâmetros da rede. É a partir daqui que o otimizador "sabe" quais variáveis ele pode alterar.

Usaremos o otimizador Adam com uma taxa de aprendizado inicial de 0.01.

O Agendador serve para alterar parâmetros do otimizador ao longo da execução. Neste exemplo, iremos diminuir a taxa de aprendizado. Ela será multiplicada por 0.95 a cada 100 épocas.

O valor de alpha serve para equilibrar as perdas de contorno com as perdas de equação. Ele será definido como 0.1.

In [ ]:
# PARA VOCÊ FAZER (1 LINHA)
# Definicao do otimizador, do agendador e de alpha
# -------------------------------
otimizador = 
agendador = 
# -------------------------------
# ATÉ AQUI

# Criar tensores e transferir para GPU

No PyTorch, as variáveis devem ser armazenadas em tensores.

Além disso, se formos rodar em GPU, precisamos manualmente carregar a rede a as variáveis na memória do GPU.

Note a opção "requires_grad" que está ativa na variável X_equação, isso sinaliza que o PyTorch deverá manter a trilha de todas as operações feitas a partir desta variável para, depois, conseguir usar a regra da cadeia e calcular as derivadas.



In [ ]:
X_equacao = torch.tensor(X_equacao,requires_grad=True,dtype=torch.float)
X_contorno = torch.tensor(X_contorno,dtype=torch.float)
Y_contorno = torch.tensor(Y_contorno,dtype=torch.float)

device = torch.device('cuda' if torch.cuda.is_available () else 'cpu')
X_equacao = X_equacao.to(device)
X_contorno = X_contorno.to(device)
Y_contorno = Y_contorno.to(device)
rna = rna.to(device)

# Testar modelo

Neste seção, vamos varificar se o modelo foi construido corretamente e preparar funções para ver os resulados

## Exercício 7 - Rodar alguns passos de otimização

Primeiro, vamos rodar 10 épocas

In [ ]:
# Colocar rede em modo de treinamento
rna.train()

# FAZER ITERAÇÃO
for epoca in range(10):

  # Resortear pontos
  if ressorteio:
    X_equacao = gerar_pontos_equacao(pontos_no_dominio,comprimento_x,comprimento_y)
    X_equacao = torch.tensor(X_equacao,requires_grad=True,dtype=torch.float).to(device)

  # PARA VOCÊ FAZER (1 LINHA)
  # Inicializar gradientes
  # -------------------------------
  
  # -------------------------------
  # ATÉ AQUI

  # PARA VOCÊ FAZER (1 LINHA)
  # Calcular perdas
  # -------------------------------
  perda, perda_contorno, perda_equacao = 
  # -------------------------------
  # ATÉ AQUI

  # PARA VOCÊ FAZER (1 LINHA)
  # Backpropagation
  # -------------------------------

  # -------------------------------
  # ATÉ AQUI

  # PARA VOCÊ FAZER (2 LINHAS)
  # Passo do otimizador e do agendador
  # -------------------------------

  # -------------------------------
  # ATÉ AQUI

  # Mostrar resultados
  print(f'Epoca: {epoca}, Perda: {perda.item()} (Contorno: {perda_contorno.item()}, Equacao: {perda_equacao.item()})')

Resultado esperado: (Note que alguma variação neste passo é possível)
```
Epoca: 0, Perda: 0.41967514157295227 (Contorno: 0.4659286439418793, Equacao: 0.003393700812011957)
Epoca: 1, Perda: 0.32494547963142395 (Contorno: 0.3601118326187134, Equacao: 0.008448254317045212)
Epoca: 2, Perda: 0.3115726113319397 (Contorno: 0.3426373302936554, Equacao: 0.03199015185236931)
Epoca: 3, Perda: 0.32265809178352356 (Contorno: 0.3523457944393158, Equacao: 0.055468954145908356)
Epoca: 4, Perda: 0.30739280581474304 (Contorno: 0.3336279094219208, Equacao: 0.07127691805362701)
Epoca: 5, Perda: 0.2841506898403168 (Contorno: 0.30777376890182495, Equacao: 0.07154306024312973)
Epoca: 6, Perda: 0.2791939973831177 (Contorno: 0.3005773425102234, Equacao: 0.0867437869310379)
Epoca: 7, Perda: 0.29101112484931946 (Contorno: 0.3120240271091461, Equacao: 0.10189513117074966)
Epoca: 8, Perda: 0.2968086898326874 (Contorno: 0.3161880671977997, Equacao: 0.12239435315132141)
Epoca: 9, Perda: 0.28874772787094116 (Contorno: 0.3070990741252899, Equacao: 0.12358570098876953)
```

## Exibir resultados

Agora, vamos preparar uma função que calcula a rna em um grid e outra que plota a solução

In [ ]:
def calcular_grid(rna, comprimento_x, comprimento_y, nx=101, ny=101):

    # Definir grid
    x = np.linspace(0.,comprimento_x,nx)
    y = np.linspace(0.,comprimento_y,ny)
    [y_grid, x_grid] = np.meshgrid(y,x)
    x = torch.tensor(x_grid.flatten()[:,None],requires_grad=True,dtype=torch.float).to(device)
    y = torch.tensor(y_grid.flatten()[:,None],requires_grad=True,dtype=torch.float).to(device)

    # Avaliar modelor
    rna.eval()
    Y_pred = rna(torch.hstack((x,y)))

    # Formatar resultados em array
    z_pred = Y_pred.cpu().detach().numpy()[:,0].reshape(x_grid.shape)

    return x_grid, y_grid, z_pred

In [ ]:
# Calcular valores da função e gerar grids
x_grid, y_grid, z_pred = calcular_grid(rna, comprimento_x, comprimento_y)

# Plotar figura
fig = go.Figure(data=[go.Surface(x=x_grid, y=y_grid, z=z_pred)])

fig.update_layout(scene=dict(aspectratio=dict(x=1.5, y=1.5, z=0.5)))

## Treinamento da rede
Agora vamos treinar o modelo por 10000 épocas

In [ ]:
numero_de_epocas = 10000
perda_historico = np.zeros(numero_de_epocas)
perda_contorno_historico = np.zeros(numero_de_epocas)
perda_equacao_historico = np.zeros(numero_de_epocas)
epocas = np.array(range(numero_de_epocas))

# Colocar rede em modo de treinamento
rna.train()

# FAZER ITERAÇÃO
for epoca in epocas:

  # Resortear pontos
  if ressorteio:
    X_equacao = gerar_pontos_equacao(pontos_no_dominio,comprimento_x,comprimento_y)
    X_equacao = torch.tensor(X_equacao,requires_grad=True,dtype=torch.float).to(device)

  # Inicializar gradientes
  otimizador.zero_grad()

  # Calcular perdas
  perda, perda_contorno, perda_equacao = calc_perda(rna,X_contorno,Y_contorno,X_equacao,alpha=alpha)

  # Backpropagation
  perda.backward()

  # Passo do otimizador
  otimizador.step()
  agendador.step()

  # Guardar logs
  perda_historico[epoca] = perda.item()
  perda_contorno_historico[epoca] = perda_contorno.item()
  perda_equacao_historico[epoca] = perda_equacao.item()

  if epoca%500==0:
    print(f'Epoca: {epoca}, Perda: {perda.item()} (Contorno: {perda_contorno.item()}, Equacao: {perda_equacao.item()})')



Saída esperada:
```
Epoca: 0, Perda: 0.2796975076198578 (Contorno: 0.29735538363456726, Equacao: 0.1207764521241188)
Epoca: 500, Perda: 0.019234932959079742 (Contorno: 0.01609576679766178, Equacao: 0.04748743027448654)
Epoca: 1000, Perda: 0.008779237978160381 (Contorno: 0.007192874792963266, Equacao: 0.023056507110595703)
Epoca: 1500, Perda: 0.004627911373972893 (Contorno: 0.004134506918489933, Equacao: 0.00906855147331953)
Epoca: 2000, Perda: 0.003794528543949127 (Contorno: 0.002829557517543435, Equacao: 0.012479268014431)
Epoca: 2500, Perda: 0.00246912962757051 (Contorno: 0.0022384794428944588, Equacao: 0.004544983152300119)
Epoca: 3000, Perda: 0.0022056633606553078 (Contorno: 0.0019008198287338018, Equacao: 0.004949256312102079)
Epoca: 3500, Perda: 0.0018194746226072311 (Contorno: 0.001648333971388638, Equacao: 0.00335973990149796)
Epoca: 4000, Perda: 0.0023012394085526466 (Contorno: 0.0015506889903917909, Equacao: 0.009056192822754383)
Epoca: 4500, Perda: 0.0015365986619144678 (Contorno: 0.0013659244868904352, Equacao: 0.0030726653058081865)
Epoca: 5000, Perda: 0.0013851739931851625 (Contorno: 0.00124174274969846, Equacao: 0.0026760550681501627)
Epoca: 5500, Perda: 0.0012605079682543874 (Contorno: 0.0011513156350702047, Equacao: 0.002243238966912031)
Epoca: 6000, Perda: 0.001264510559849441 (Contorno: 0.001088821212761104, Equacao: 0.00284571573138237)
Epoca: 6500, Perda: 0.001118625863455236 (Contorno: 0.0010235330555588007, Equacao: 0.0019744618330150843)
Epoca: 7000, Perda: 0.001104312133975327 (Contorno: 0.0009742183610796928, Equacao: 0.0022751560900360346)
Epoca: 7500, Perda: 0.0011349612614139915 (Contorno: 0.0009340505348518491, Equacao: 0.0029431579168885946)
Epoca: 8000, Perda: 0.00105355738196522 (Contorno: 0.0008986592874862254, Equacao: 0.002447640523314476)
Epoca: 8500, Perda: 0.0009644646197557449 (Contorno: 0.0008754738955758512, Equacao: 0.0017653811955824494)
Epoca: 9000, Perda: 0.0009447543416172266 (Contorno: 0.0008521503186784685, Equacao: 0.0017781907226890326)
Epoca: 9500, Perda: 0.0009573057177476585 (Contorno: 0.000830349454190582, Equacao: 0.002099912380799651)
```

In [ ]:
# Calcular valores da função e gerar grids
x_grid, y_grid, z_pred = calcular_grid(rna, comprimento_x, comprimento_y)

# Plotar figura
fig = go.Figure(data=[go.Surface(x=x_grid, y=y_grid, z=z_pred)])

fig.update_layout(scene=dict(aspectratio=dict(x=1.5, y=1.5, z=0.5)))

In [ ]:
# Plotar histórico
fig = go.FigureWidget()
fig.add_trace(go.Scatter(x=epocas, y=perda_historico, name='Total', line=dict(color='black', width=4)))
fig.add_trace(go.Scatter(x=epocas, y=perda_contorno_historico, name='Contorno', line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=epocas, y=perda_equacao_historico, name='Equacao', line=dict(color='red', width=2)))
fig.update_yaxes(type="log")
fig.show(renderer="colab")

# Sugestão:
Tente alterar as condições de contorno para observar se o PINN consegue encontrar as soluções.

Tente introduzir um termo forçante na equação, algo como:

$\frac{\partial^2 z}{\partial x^2} + \frac{\partial^2 z}{\partial y^2} = f(x,y)$

Veja o efeito de usar RNAs mais simples ou mais complexas.